In [0]:
# Mount google drive to this environment

from google.colab import drive
drive.mount('drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at drive


In [0]:
# Read all the tweets file

import pandas as pd

tweet_data = pd.DataFrame()

dir = 'drive/My Drive/data/russ_twit/'

for i in range(1,10):
  temp = pd.read_csv(dir + 'IRAhandle_tweets_' + str(i) + '.csv')
  tweet_data = tweet_data.append(temp)
  
tweet_data.head()

,external_author_id,author,content,region,language,publish_date,harvested_date,following,followers,updates,post_type,account_type,new_june_2018,retweet,account_category
0,9.060000e+17,10_GOP,"""We have a sitting Democrat US Senator on tria...",Unknown,English,10/1/2017 19:58,10/1/2017 19:59,1052,9636,253,NaN,Right,0,0,RightTroll
1,9.060000e+17,10_GOP,Marshawn Lynch arrives to game in anti-Trump s...,Unknown,English,10/1/2017 22:43,10/1/2017 22:43,1054,9637,254,NaN,Right,0,0,RightTroll
2,9.060000e+17,10_GOP,Daughter of fallen Navy Sailor delivers powerf...,Unknown,English,10/1/2017 22:50,10/1/2017 22:51,1054,9637,255,RETWEET,Right,0,1,RightTroll
3,9.060000e+17,10_GOP,JUST IN: President Trump dedicates Presidents ...,Unknown,English,10/1/2017 23:52,10/1/2017 23:52,1062,9642,256,NaN,Right,0,0,RightTroll
4,9.060000e+17,10_GOP,"19,000 RESPECTING our National Anthem! #StandF...",Unknown,English,10/1/2017 2:13,10/1/2017 2:13,1050,9645,246,RETWEET,Right,0,1,RightTroll


15 fields in this datafram. For this project we focus on content, language, and publish_data. 

In [0]:
# Process english only tweets

eng_only = tweet_data[tweet_data['language'] == 'English']

eng_only.head()

,external_author_id,author,content,region,language,publish_date,harvested_date,following,followers,updates,post_type,account_type,new_june_2018,retweet,account_category
0,9.060000e+17,10_GOP,"""We have a sitting Democrat US Senator on tria...",Unknown,English,10/1/2017 19:58,10/1/2017 19:59,1052,9636,253,NaN,Right,0,0,RightTroll
1,9.060000e+17,10_GOP,Marshawn Lynch arrives to game in anti-Trump s...,Unknown,English,10/1/2017 22:43,10/1/2017 22:43,1054,9637,254,NaN,Right,0,0,RightTroll
2,9.060000e+17,10_GOP,Daughter of fallen Navy Sailor delivers powerf...,Unknown,English,10/1/2017 22:50,10/1/2017 22:51,1054,9637,255,RETWEET,Right,0,1,RightTroll
3,9.060000e+17,10_GOP,JUST IN: President Trump dedicates Presidents ...,Unknown,English,10/1/2017 23:52,10/1/2017 23:52,1062,9642,256,NaN,Right,0,0,RightTroll
4,9.060000e+17,10_GOP,"19,000 RESPECTING our National Anthem! #StandF...",Unknown,English,10/1/2017 2:13,10/1/2017 2:13,1050,9645,246,RETWEET,Right,0,1,RightTroll


In [0]:
# Get tweet text

tweet_text = eng_only['content'].astype(str).values.tolist()

# tweet_date = eng_only['publish_date'].astype(str).values.tolist()

tweet_text[4]

# tweet_date[0:3]

'19,000 RESPECTING our National Anthem! #StandForOurAnthem🇺🇸 https://t.co/czutyGaMQV'

In [0]:
import re

import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

from nltk.stem import PorterStemmer


english_stopwords = stopwords.words('english')

tweet_words = []

stemmer = PorterStemmer()

for tweet in tweet_text:

  # Remove @ handle and letters after it
  twit_temp = re.sub(r'@(\w+)', '', tweet.lower())

  # Remove # hashtag but keep letters after it
  twit_temp = re.sub(r'#(\w+)', r'\g<1>', twit_temp)

  # Remove all the links
  twit_temp = re.sub(r'(http|https|ftp)://[a-zA-Z0-9\\./]+', '', twit_temp)

  # Remove all the emoji
  emoji_pattern = re.compile("["
                         u"\U0001F600-\U0001F64F"  # emoticons
                         u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                         u"\U0001F680-\U0001F6FF"  # transport & map symbols
                         u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                         "]+", flags=re.UNICODE)

  twit_temp = re.sub(emoji_pattern, '', twit_temp)

  # Remove all the punctuation
  twit_temp = re.sub(r'[^\w\s]', '', twit_temp)
  twit_temp = twit_temp.split()
  
  twit_clean = []

  # Remove stopwords and stemming the word
  for x in twit_temp:
      if x not in english_stopwords:
          twit_clean.append(stemmer.stem(x))
  
  tweet_words.append(twit_clean)
  
print(tweet_words[4])

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
['19000', 'respect', 'nation', 'anthem', 'standforouranthem']


In [0]:
import gensim

# Train our model with gensim, igore the word showed up only once
model = gensim.models.Word2Vec(tweet_words, min_count=2, workers=4)

In [0]:
# Testing 
print(model.most_similar('presid'))

[('presidentelect', 0.6994872093200684), ('pre', 0.675586462020874), ('potu', 0.6505663394927979), ('donald', 0.5883846282958984), ('administr', 0.5860335230827332), ('millenialsaim', 0.5845510363578796), ('versa', 0.5517970323562622), ('presidenti', 0.5429940223693848), ('updatedpresid', 0.539179265499115), ('prez', 0.5027735829353333)]


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [0]:
# model.save('drive/My Drive/data/russ_twit_model/myModel')

# model.wv.save("/content/gdrive/My Drive/data/russ_twit_model/myModel.wv")

# '/content/gdrive/My Drive/colab_dir/mnist_model.h5'

# Save the model
model.wv.save_word2vec_format('/content/drive/My Drive/data/russ_twit_model/myModel.txt',binary = False)

# !ls /content/drive/'My Drive'

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
